<a href="https://colab.research.google.com/github/aidyai/rss-ai/blob/main/rss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade --quiet  apify-client langchain-openai langchain
!pip install langchain_community
!pip install feedparser newspaper3k openai
!pip install openai==0.28
!pip install replicate

In [4]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.utilities import ApifyWrapper
from langchain_core.documents import Document
import feedparser
import requests
from bs4 import BeautifulSoup
import openai
from newspaper import Article

In [ ]:
import os
os.environ["APIFY_API_TOKEN"] = ""
#os.environ["OPENAI_API_KEY"] = ""

## ARTICLE EXTRACTION: This code only extracts onearticle but can be modified to work for the latest 10, 50 or 20 articles

In [ ]:
https://news.google.com/rss/search?q=umoeno+after:2017-01-01+before:2024-08-07&hl=en&gl=US&ceid=US:en


In [ ]:
https://news.google.com/rss/search?q=akwaibom+after:2017-01-01+before:2024-08-07&hl=en&gl=US&ceid=US:en


In [ ]:
https://news.google.com/rss/search?q={element}=en-US&gl=US&ceid=US%3Aen

In [ ]:
https://news.google.com/rss/search?q=Akwa Ibom Salary=en-US&gl=US&ceid=US%3Aen

In [ ]:
https://news.google.com/rss/search?q=NigerianPolitics+after:2017-01-01+before:2024-08-07&hl=en&gl=US&ceid=US:en


In [5]:
# RSS feed URL
#rss_feed_url = "https://collegevilledentistry.com/"

op = 'https://news.google.com/rss/search?q=akwa%20ibom&hl=en-US&gl=US&ceid=US%3Aen'

# Parse the RSS feed
feed = feedparser.parse(op)

# Get the latest just one articles
latest_articles = feed.entries[0:]   # we just yse one for speed

In [6]:
print(latest_articles)

[{'title': 'Akwa Ibom Governor Eno Commends Youths for Shunning Nationwide Protests, Announces Monthly Welfare For 5,000 Youths - Arise News', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://news.google.com/rss/search?q=akwa%20ibom&hl=en-US&gl=US&ceid=US%3Aen', 'value': 'Akwa Ibom Governor Eno Commends Youths for Shunning Nationwide Protests, Announces Monthly Welfare For 5,000 Youths - Arise News'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://news.google.com/rss/articles/CBMi0AFBVV95cUxQWWs3T0F3U1kyV2dRS3E3XzFZc0NwMkl3bjVjcGxXdEcydVhqeVByd0VqNVpZY01jZ0UzVUFlajlVbFFJMENGbEFQZ2h4M3l3NU1CYnh2MTRlLWgtc1dnUlN2MWJFZFhabE82SmM0UUp1ZFNaTXlJNE5IYUxUaVk2UmhBZmx2cDJ5RTJYbWJiLW91MGk3ZWFIMzNQNmFBYnZSakc4VWR1X1JsbWhzcDU4aHdPbnFKcFFPbW9YMmdtdy1JSXQyLTVDTnczZlMwRWs2?oc=5'}], 'link': 'https://news.google.com/rss/articles/CBMi0AFBVV95cUxQWWs3T0F3U1kyV2dRS3E3XzFZc0NwMkl3bjVjcGxXdEcydVhqeVByd0VqNVpZY01jZ0UzVUFlajlVbFFJMENGbEFQZ2h4M3l3NU1CYnh2MTRlLWgtc1dn

In [7]:
elements = [
    "Bad Roads", "Unemployment", "Infrastructure", "Economic Crisis/Poverty", "Insecurity",
    "Flooding", "Politics/Election", "Weather Forecast", "Education", "Money Laundering",
    "Health", "War/Crime", "Fashion Trends", "Technology", "Power Supply", "Agriculture",
    "Fraud", "Religion", "Entertainment", "Military", "Foreign Exchange (Dollar/Naira)",
    "Trading", "Scarcity", "Industries", "Business", "Export/Import", "POS", "Senate",
    "Busary", "Budget", "Governor Umo Eno", "Governor Udom Emmanuel", "Akwa Ibom Senate",
    "Akwa Ibom Political Party", "Akwa Ibom Free Education", "Akwa Ibom Civil Servant",
    "Akwa Ibom Salary", "Bursary Akwa Ibom", "Church Akwa Ibom", "Market Akwa Ibom",
    "Food Vendors", "POS Akwa Ibom", "Akwa Ibom Farming", "Akwa Ibom Elections", "Akwa Ibom Dakkada",
    "Abia", "Adamawa", "Akwa Ibom", "Anambra", "Bauchi", "Bayelsa", "Benue", "Borno",
    "Cross River", "Delta", "Ebonyi", "Edo", "Ekiti", "Enugu", "FCT - Abuja", "Gombe",
    "Imo", "Jigawa", "Kaduna", "Kano", "Katsina", "Kebbi", "Kogi", "Kwara", "Lagos",
    "Nasarawa", "Niger", "Ogun", "Ondo", "Osun", "Oyo", "Plateau", "Rivers", "Sokoto",
    "Taraba", "Yobe", "Zamfara"
]

base_url = "https://news.google.com/rss/search?q={element}=en-US&gl=US&ceid=US%3Aen"

urls = [base_url.format(element=element.replace(" ", "+")) for element in elements]


In [9]:
print(len(urls))

82


In [10]:
def extract_links_from_rss(url):
    feed = feedparser.parse(url)
    links = [entry.link for entry in feed.entries]
    return links

# Extract links from all RSS feeds
all_links = []
for url in urls:
    all_links.extend(extract_links_from_rss(url))

In [11]:
print(len(all_links))

4507


In [ ]:
apify = ApifyWrapper()

def fetch_articles(url):
    loader = apify.call_actor(
        actor_id="apify/website-content-crawler",
        run_input={"startUrls": [{"url": url}]},
        dataset_mapping_function=lambda item: Document(
            page_content=item["text"] or "", metadata={"source": item["url"]}
        ),
    )
    return loader.load()

In [ ]:
# Fetch articles from all links
articles = []
for link in all_links:
    articles.extend(fetch_articles(link))

# Now `articles` contains the content fetched from all links


In [ ]:
this is LINK

-------FOR ENGLISH:https://www.bible.com/bible/1/REV.21.KJV

-------FOR IBIBIO:https://www.bible.com/bible/2619/REV.21.NAAM

-------FOR EFIK;https://www.bible.com/bible/2570/REV.21.ENA


VERSE = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
CHAPTER = MRK
VERSION = ["KJV", "NAAM", "ENA"]


https://www.bible.com/bible/1/{MRK}.{1}.{KJV}
https://www.bible.com/bible/2619/{MRK}.{16}.{NAAM}
https://www.bible.com/bible/2570/{MRK}.{1}.{ENA}

In [ ]:
apify = ApifyWrapper()

In [ ]:
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": [{"url": "https://collegevilledentistry.com/General-Dentistry"}]},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

In [ ]:
apify = ApifyWrapper()

loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": [{"url": url}]},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

article_text = loader.load()

In [ ]:
article_text = loader.load()

In [ ]:
article_text

[Document(metadata={'source': 'https://collegevilledentistry.com/General-Dentistry'}, page_content='Collegeville, PA General Dentistry l Advanced Dentistry of Collegeville\nAre you looking for a caring dentist willing to spend the time with your entire family? Dr. Silva provides the highest level of care through careful listening, understanding, and communication. We strive to make every patient’s visit comfortable and relaxing.\nTo keep your teeth and mouth in optimal condition, you need to visit us for checkups and cleanings every six months. Dr. Silva can help you ward off problems like gum disease, which has been found to contribute to a host of overall health issues. Also, when tooth decay is treated in the early stage, the procedure is minimally invasive and may not even require anesthesia. Left to spread, gum disease and tooth decay can destroy a healthy smile.\nYour checkup will include digital X-rays (if necessary), an oral cancer screening, bite assessment, and tooth-by-tooth

In [ ]:
loader

ApifyDatasetLoader(apify_client=<apify_client.client.ApifyClient object at 0x7dd150978be0>, dataset_id='8X7Yg5DnMzw6SpWES', dataset_mapping_function=<function <lambda> at 0x7dd1503f4310>)

In [ ]:
title = []
urls_to_crawl = []

for article in latest_articles:

    # extracting the title and article URL
    title = article.title
    article_url = article.link

    # Appending the article title and URL
    #title.append({"title": title})
    urls_to_crawl.append({"url": article_url})

In [ ]:
print(urls_to_crawl)

[{'url': 'https://news.google.com/rss/articles/CBMiXGh0dHBzOi8vY2hhbXBpb25uZXdzLmNvbS5uZy9ha3BhYmlvLXRvLWVuby15b3UtaGF2ZS1hLWRpdmluZS1kdXR5LXRvLXVuaXRlLWFrd2EtaWJvbS1wZW9wbGUv0gEA?oc=5'}]


In [ ]:
links = ["https://collegevilledentistry.com/General-Dentistry",
    "https://collegevilledentistry.com/Laser-Dentistry",
    "https://collegevilledentistry.com/Cosmetic-Laser-Treatment",
    "https://collegevilledentistry.com/Restorative-Dentistry"
    "https://collegevilledentistry.com/Implant-Dentistry",
    "https://collegevilledentistry.com/surgery-post-op-instructions/",
    "https://collegevilledentistry.com/new-denture-instructions/",
    "https://collegevilledentistry.com/Surgery-Post-Op-Instructions/",
    "https://collegevilledentistry.com/New-Denture-Instructions/",
    "https://collegevilledentistry.com/Whitening-For-Life/",
    "https://collegevilledentistry.com/Invisalign/",
    "https://collegevilledentistry.com/Sedation-Dentistry/",
    "https://collegevilledentistry.com/conscious-sedation/",
    "https://collegevilledentistry.com/Periodontics",
    "https://collegevilledentistry.com/Tooth-Extraction/",
    "https://collegevilledentistry.com/virtual-smile-consultation/",
    "https://collegevilledentistry.com/veneers/",
    "https://collegevilledentistry.com/cosmetic/",
    "https://collegevilledentistry.com/dermal-fillers/",
    "https://collegevilledentistry.com/botox/",
    "https://collegevilledentistry.com/facial-enhancement/",
    "https://collegevilledentistry.com/teeth-Whitening",
    "https://collegevilledentistry.com/kor-whitening/"
]

In [ ]:
start_urls = [{"url": url} for url in links]

In [ ]:
#this functions looks into the urls_to_crawl and extracts its text
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
   run_input={"startUrls": [{"url": "https://collegevilledentistry.com/General-Dentistry"}]},

    run_input={"startUrls": links},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

In [ ]:
#this functions looks into the urls_to_crawl and extracts its text
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": start_urls},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

In [ ]:
# Call the actor with the formatted list of URLs
loader = apify_client.actor("apify/website-content-crawler").call(
    run_input={
        "startUrls": start_urls
    },
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "",
        metadata={"source": item["url"]}
    ),
)

# Process the loader result as needed
documents = loader.items
for document in documents:
    print(document.page_content)
    print(document.metadata)

In [ ]:
print(len(links))

23


In [ ]:
#this functions looks into the urls_to_crawl and extracts its text
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": start_urls},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

# BEAUTIFUL SOUP

In [ ]:
link = "https://collegevilledentistry.com/General-Dentistry"

scrape_content(link)

Content from https://collegevilledentistry.com/General-Dentistry:

Skip to content
(610) 489-5555
Pay Now
Facebook page opens in new window
Twitter page opens in new window
YouTube page opens in new window
Instagram page opens in new window
Collegeville Dentistry
Home
About Us
About Us
Meet Dr. Silva
Meet Our Team
Testimonials
Events
Advanced Technology
iTero® Element™ Scanner.
TENS & K7
the Wand Advanced Anesthesia
Vatech Ctscan
Identafi® Oral Cancer Screening
Employment Opportunities
Treatments
General Dentistry
Laser Dentistry
Cosmetic Laser Treatment
Restorative Dentistry
Implant Dentistry
Patient Education
Surgery Post Op Instructions
New Denture Instructions
Whitening For Life
Invisalign
Sedation Dentistry
Anxiolysis “Conscious sedation”
Periodontics
Oral Surgery / Tooth Extraction
Cosmetic Dentistry
Virtual Smile Consultation
Veneers / Smile Makeover
Cosmetic Dentistry Before & After
Dermal Fillers
Botox
Ageless Dentistry
Facial Enhancement / Full Mouth Reconstruction
Zoom! Teet

In [ ]:
import requests
from bs4 import BeautifulSoup

# List of URLs to scrape
links = [
    "https://collegevilledentistry.com/General-Dentistry",
    "https://collegevilledentistry.com/Laser-Dentistry",
    "https://collegevilledentistry.com/Cosmetic-Laser-Treatment",
    "https://collegevilledentistry.com/Restorative-Dentistry",
    "https://collegevilledentistry.com/Implant-Dentistry",
    "https://collegevilledentistry.com/surgery-post-op-instructions/",
    "https://collegevilledentistry.com/new-denture-instructions/",
    "https://collegevilledentistry.com/Surgery-Post-Op-Instructions/",
    "https://collegevilledentistry.com/New-Denture-Instructions/",
    "https://collegevilledentistry.com/Whitening-For-Life/",
    "https://collegevilledentistry.com/Invisalign/",
    "https://collegevilledentistry.com/Sedation-Dentistry/",
    "https://collegevilledentistry.com/conscious-sedation/",
    "https://collegevilledentistry.com/Periodontics",
    "https://collegevilledentistry.com/Tooth-Extraction/",
    "https://collegevilledentistry.com/virtual-smile-consultation/",
    "https://collegevilledentistry.com/veneers/",
    "https://collegevilledentistry.com/cosmetic/",
    "https://collegevilledentistry.com/dermal-fillers/",
    "https://collegevilledentistry.com/botox/",
    "https://collegevilledentistry.com/facial-enhancement/",
    "https://collegevilledentistry.com/teeth-Whitening",
    "https://collegevilledentistry.com/kor-whitening/"
]

# Function to scrape content from a URL
def scrape_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # Extract the main content from the page
        # Adjust the selector according to the structure of the website
        main_content = soup.find('main') or soup.find('body')
        if main_content:
            text_content = main_content.get_text(separator='\n', strip=True)
            print(f"Content from {url}:\n")
            print(text_content)
            print("\n" + "="*80 + "\n")
        else:
            print(f"No main content found for {url}")
    else:
        print(f"Failed to retrieve {url} (status code: {response.status_code})")

# Iterate over the list of URLs and scrape each one
for link in links:
    scrape_content(link)

Content from https://collegevilledentistry.com/General-Dentistry:

Skip to content
(610) 489-5555
Pay Now
Facebook page opens in new window
Twitter page opens in new window
YouTube page opens in new window
Instagram page opens in new window
Collegeville Dentistry
Home
About Us
About Us
Meet Dr. Silva
Meet Our Team
Testimonials
Events
Advanced Technology
iTero® Element™ Scanner.
TENS & K7
the Wand Advanced Anesthesia
Vatech Ctscan
Identafi® Oral Cancer Screening
Employment Opportunities
Treatments
General Dentistry
Laser Dentistry
Cosmetic Laser Treatment
Restorative Dentistry
Implant Dentistry
Patient Education
Surgery Post Op Instructions
New Denture Instructions
Whitening For Life
Invisalign
Sedation Dentistry
Anxiolysis “Conscious sedation”
Periodontics
Oral Surgery / Tooth Extraction
Cosmetic Dentistry
Virtual Smile Consultation
Veneers / Smile Makeover
Cosmetic Dentistry Before & After
Dermal Fillers
Botox
Ageless Dentistry
Facial Enhancement / Full Mouth Reconstruction
Zoom! Teet

# Extracting the title, article_url and article_text for the latest 10 articles

In [ ]:
print(title)                      #EXTRACTED TITLE
print(urls_to_crawl)              #EXTRACTED URL

article_text = loader.load()
article_text                      #EXTRACTED ARTICLE TEXY

Akpabio To Eno: You have a divine duty to unite Akwa Ibom People - Champion Newspapers
[{'url': 'https://news.google.com/rss/articles/CBMiXGh0dHBzOi8vY2hhbXBpb25uZXdzLmNvbS5uZy9ha3BhYmlvLXRvLWVuby15b3UtaGF2ZS1hLWRpdmluZS1kdXR5LXRvLXVuaXRlLWFrd2EtaWJvbS1wZW9wbGUv0gEA?oc=5'}]


[Document(page_content='President of the Senate, Godswill Akpabio has charged the Akwa Ibom State Governor, Pastor Umo Eno, on the need to unite the state across party lines for the continued development and progress of the state and not make his father’s enemies his enemies.\nHe described the Governor as a man of peace and an Apostle and disciple of peace, noting, “Akwa Ibom is a very easy state to rule and govern but also a very difficult place to unite.”\nAkpabio gave the charge while on a courtesy visit to the Governor in his office, in Uyo on Friday.\nA statement by his Special Assistant on Media, Jackson Udom, quoted the Senate President as saying, “it is a major duty bestowed on you by God to unite the whole state. Let your actions speak unity to the the state. As a governor, I united the state by appointing people into key positions without considering where they came from.\n“I want you to unite your people without any political considerations. Even if you inherited a policy th

In [ ]:
import json
# Extracting content from the list
extracted_content = [document.page_content for document in article_text]

In [ ]:
extracted_content

['President of the Senate, Godswill Akpabio has charged the Akwa Ibom State Governor, Pastor Umo Eno, on the need to unite the state across party lines for the continued development and progress of the state and not make his father’s enemies his enemies.\nHe described the Governor as a man of peace and an Apostle and disciple of peace, noting, “Akwa Ibom is a very easy state to rule and govern but also a very difficult place to unite.”\nAkpabio gave the charge while on a courtesy visit to the Governor in his office, in Uyo on Friday.\nA statement by his Special Assistant on Media, Jackson Udom, quoted the Senate President as saying, “it is a major duty bestowed on you by God to unite the whole state. Let your actions speak unity to the the state. As a governor, I united the state by appointing people into key positions without considering where they came from.\n“I want you to unite your people without any political considerations. Even if you inherited a policy that was not in the best

In [ ]:
import openai
# Send the content to OpenAI to get topic and category tags
response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=extracted_content,
    max_tokens=50,  # Adjust as needed
    n=1,  # Number of completions to generate
    stop=["\n"],  # Stop completion at the end of the text
    temperature=0.5,  # Adjust as needed
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    logit_bias={},
    logprobs=0,
    echo=True  # Print the input prompt along with the completion
)


# USING REPLICATE API TO CALL POWERFUL LARGE LANGUAGE MODELS

In [ ]:
import replicate

In [ ]:
import os
import replicate
from replicate.client import Client

replicate = Client(api_token="")

## USING MISTRAL 8x7B via Replicate Api

In [ ]:
for event in replicate.stream(
    "mistralai/mixtral-8x7b-instruct-v0.1",
    input={
        "top_k": 50,
        "top_p": 0.9,
        "prompt": f"Generate 5 'TOPIC TAGS' AND 'CATEGPRY TAGS' for the given article\ {extracted_content}",
        "temperature": 0.6,
        "system_prompt": "YOU ARE AN AI ASSIATAT THAT CAN GENERATE topic tags AND  article category tags",
        "length_penalty": 1,
        "max_new_tokens": 1024,
        "prompt_template": "<s>[INST] {prompt} [/INST] ",
        "presence_penalty": 0
    },
):
    print(str(event), end="")

ReplicateError: ReplicateError Details:
title: Free time limit reached
status: 402
detail: You have reached the free time limit. To continue using Replicate, set up billing at https://replicate.com/account/billing#billing.

## USING METAS LLAMA 3 Via Replicate Api

In [ ]:
for event in replicate.stream(
    "meta/meta-llama-3-70b-instruct",
    input={
        "top_k": 0,
        "top_p": 0.9,
        "prompt": f"Generate 5 'TOPIC TAGS' AND 'CATEGPRY TAGS' for the given article\ {extracted_content}",
        "max_tokens": 650,
        "min_tokens": 0,
        "temperature": 0.6,
        "system_prompt": "YOU ARE AN AI ASSITANT THAT CAN GENERATE topic tags AND article category tags",
        "length_penalty": 1,
        "stop_sequences": "<|end_of_text|>,<|eot_id|>",
        "prompt_template": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
        "presence_penalty": 1.15,
        "log_performance_metrics": False
    },
):
    print(str(event), end="")

Here are five topic tags and five category tags for the given article:

**Topic Tags:**

1. Oil Production
2. Nigeria Energy Sector
3. NNPC E&P Ltd
4. Natural Oilfield Services Ltd
5. Oil Mining Lease (OML) 13

**Category Tags:**

1. Business
2. Energy
3. Nigeria News
4. Oil and Gas
5. Economy

## RECIEVES NOTIFICATION

In [ ]:
!pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00


In [ ]:
import replicate
replicate_api_key = ""

In [ ]:
# Set the API key
rep_api = replicate.Client(api_token=replicate_api_key)

In [ ]:
user_input = "I want to create content around the various ways my Football Agency can get recognition and partnerships"

json_template = """
{
    "storri_intro": {
        "inspiration_metadata": {
            "heading": "Introduction Heading"
        }
    },
    "storri_body": {
        "storri_body_1": {
            "inspiration_metadata": {
                "heading": "First Section Heading",
                "talking_points": [
                    "First talking point related to the heading",
                    "Second talking point related to the heading",
                    "Third talking point related to the heading"
                ],
                "media": [
                    "First media prompt for image generation",
                    "Second media prompt for image generation",
                    "Third media prompt for video generation",
                    "Fourth media prompt for video generation"
                ]
            }
        },
        "storri_body_2": {
            "inspiration_metadata": {
                "heading": "Second Section Heading",
                "talking_points": [
                    "First talking point related to the heading",
                    "Second talking point related to the heading",
                    "Third talking point related to the heading"
                ],
                "media": [
                    "First media prompt for image generation",
                    "Second media prompt for image generation",
                    "Third media prompt for video generation",
                    "Fourth media prompt for video generation"
                ]
            }
        },
        "storri_body_3": {
            "inspiration_metadata": {
                "heading": "Third Section Heading",
                "talking_points": [
                    "First talking point related to the heading",
                    "Second talking point related to the heading",
                    "Third talking point related to the heading"
                ],
                "media": [
                    "First media prompt for image generation",
                    "Second media prompt for image generation",
                    "Third media prompt for video generation",
                    "Fourth media prompt for video generation"
                ]
            }
        }
    }
}
"""

prompt = f"<s>[INST] {user_input} [/INST]"

system_prompt = (
    "YOU ARE AN AI ASSISTANT THAT CAN generate a JSON structure based on user input. The structure should include: "
    "1) An introduction heading (10-15 words). "
    "2) Three sections in the body, each with a heading (10-15 words), three talking points, and four media prompts suitable for generating both images and videos. "
    "The output should follow this JSON format:\n\n"
    "Each section has a heading (10-15 words), three detailed talking points, and corresponding media descriptions for generating relevant images and videos"
    'The "media" field contains three prompts that can be used to generate either images or videos, ensuring versatility in content creation.'
    'The talking points should be anything in the realm of similar words to “talking”, “discuss”, “Clarify”, “Present”,” Reveal”. It should be words that seems to want to talk that’s why it is called “talking point”.'
    f"{json_template}"
)

In [ ]:
for event in rep_api.stream(
    "mistralai/mixtral-8x7b-instruct-v0.1",
    input={
        "top_k": 50,
        "top_p": 0.9,
        "prompt": prompt,
        "temperature": 0.6,
        "system_prompt": system_prompt,
        "length_penalty": 1,
        "max_new_tokens": 1024,
        "presence_penalty": 0
    },
):
    print(str(event), end="")


Here is a JSON structure based on your input:

{
    "storri\_intro": {
        "inspiration\_metadata": {
            "heading": "Introduction to Football Agency Recognition and Partnerships"
        }
    },
    "storri\_body": {
        "storri\_body\_1": {
            "inspiration\_metadata": {
                "heading": "Maximizing Social Media Presence",
                "talking\_points": [
                    "Utilizing popular platforms like Instagram, Twitter, and Facebook to showcase successful client stories and agency achievements",
                    "Sharing behind-the-scenes content to create a personal connection with followers and potential partners",
                    "Collaborating with influencers and other brands to expand reach and credibility"
                ],
                "media": [
                    "Images of football players with agency branding on social media",
                    "Screenshots of engaging social media posts with high follower inte

In [ ]:
prompt = f"<s>[INST] {user_input} [/INST]"

system_prompt = (
    "You are an AI assistant that generates a JSON structure based on user input. The structure should include: "
    "1) An introduction heading (10-15 words). "
    "2) Three sections in the body, each with a heading (10-15 words), three talking points phrased as sub-topics "
    "instructing to discuss specific aspects of the main heading, and four media prompts suitable for generating both images and videos. "
    "The talking points should provide clear, specific instructions related to the heading. "
    "The media prompts should be vivid and detailed, suitable for generating high-quality images and videos. "
    "The output should follow this JSON format:\n\n"
    f"{json_template}"
)

for event in rep_api.stream(
    "mistralai/mixtral-8x7b-instruct-v0.1",
    input={
        "top_k": 50,
        "top_p": 0.9,
        "prompt": prompt,
        "temperature": 0.6,
        "system_prompt": system_prompt,
        "length_penalty": 1,
        "max_new_tokens": 1024,
        "presence_penalty": 0
    },
):
    print(str(event), end="")

In [ ]:
for event in rep_api.stream(
    "mistralai/mixtral-8x7b-instruct-v0.1",
    input={
        "top_k": 50,
        "top_p": 0.9,
        "prompt": ""
        "prompt": f"<s>[INST]  {user_input} [/INST]",
        "temperature": 0.6,
        "system_prompt": "YOU ARE AN AI ASSISTANT THAT CAN generate An heading (10-15 word line), five (3) talking_points, a media description (image & video) ",
        "length_penalty": 1,
        "max_new_tokens": 1024,
        "presence_penalty": 0
    },
):
    print(str(event), end="")

In [ ]:
# Example user input
user_input = "Today I want to talk about the various ways I can expand my Fruit (Udara) Business."

# Prompt template
prompt_template = """
{
  "user_input": "{user_input}",
  "output": {
    "heading": "",
    "talking_points": [],
    "recommended_image_description": "",
    "recommended_video_description": ""
  }
}
"""

# Format the prompt with the actual user input
#formatted_prompt = prompt_template.format(user_input=user_input)

# API call to replicate
for event in rep_api.stream(
    "mistralai/mixtral-8x7b-instruct-v0.1",
    input={
        "top_k": 50,
        "top_p": 0.9,
        "prompt": f"<s>[INST] generate An heading, five (5) talking_points, a recommended_image_description and a recommended_video_description from this {user_input} [/INST]",
        "temperature": 0.6,
        "system_prompt": "YOU ARE AN AI ASSISTANT THAT CAN generate An heading, five (5) talking_points, a recommended_image_description and a recommended_video_description",
        "length_penalty": 1,
        "max_new_tokens": 1024,
        "presence_penalty": 0
    },
):
    print(str(event), end="")


Heading: Expanding Your Udara Fruit Business: Innovative Approaches and Strategies

Talking Points:

1. Diversifying Product Offerings: Explore new Udara fruit products such as jams, juices, and dried fruits to attract a wider customer base.
2. Tapping into Online Sales: Utilize e-commerce platforms and social media to reach a global audience and increase sales.
3. Building Strong Supplier Relationships: Establish connections with local farmers and suppliers to ensure a consistent supply of high-quality Udara fruits.
4. Implementing Sustainable Practices: Adopt eco-friendly packaging and production methods to appeal to environmentally conscious consumers.
5. Providing Excellent Customer Service: Invest in training staff to provide top-notch customer service and build customer loyalty.

Recommended Image Description: An image of a vibrant display of Udara fruits, with various value-added products such as jams, juices, and dried fruits arranged around it.

Recommended Video Description: 

In [ ]:
import replicate

# Example user input
user_input = "Today I want to talk about the various ways I can expand my Fruit (Udara) Business."

# Prompt template
prompt_template = """
{
  'intro': {
      'inspiration_metadata' : {
          'heading' : ''
      }
  },
  'body' : {
      'section_1' : {
          'inspiration_metadata' : {
              'heading' : '',
              'talking_points' : [],
              'recommended_media_description' : []
          }
      }
  }
}
"""

# API call to replicate
for event in rep_api.stream(
    "mistralai/mixtral-8x7b-instruct-v0.1",
    input={
        "top_k": 50,
        "top_p": 0.9,
        "prompt": f"<s>[INST] Generate a JSON object with an 'intro' containing an 'inspiration_metadata' dictionary with an 'heading'. Also, generate a 'body' with a 'section_1' that includes 'inspiration_metadata' containing an 'heading', five (5) 'talking_points', and three (3) 'recommended_media_description' entries based on this input: {user_input} [/INST]",
        "temperature": 0.6,
        "system_prompt": "YOU ARE AN AI ASSISTANT THAT CAN generate a JSON object with an 'intro' containing an 'inspiration_metadata' dictionary with an 'heading'. Also, generate a 'body' with a 'section_1' that includes 'inspiration_metadata' containing an 'heading', five (5) 'talking_points', and three (3) 'recommended_media_description' entries.",
        "length_penalty": 1,
        "max_new_tokens": 1024,
        "presence_penalty": 0
    },
):
    print(str(event), end="")

{
"intro": {
"inspiration\_metadata": {
"heading": "Expanding Your Fruit Business: Unlocking New Opportunities"
}
},
"body": {
"section\_1": {
"inspiration\_metadata": {
"heading": "Strategies for Growing Your Udara Business"
},
"talking\_points": [
"1. Diversify your product offerings: Consider expanding into other fruits and value-added products such as fruit juices, jams, and dried fruits.",
"2. Explore new markets: Identify potential markets, both locally and internationally, where there is a demand for Udara and other fruits.",
"3. Improve your supply chain: Optimize your supply chain by implementing efficient logistics and inventory management systems.",
"4. Leverage technology: Utilize technology to streamline operations, improve customer engagement, and gain a competitive edge.",
"5. Build strategic partnerships: Collaborate with other businesses, organizations, and influencers to expand your reach and enhance your brand's reputation."
],
"recommended\_media\_description": [
"I

In [ ]:
# Example user input
user_input = "Today I want to talk about the various ways I can expand my Fruit (Udara) Business."

# Prompt template
prompt_template = """
{
  "user_input": "{user_input}",
  "output": {
    "heading": "",
    "talking_points": [],
    "recommended_image_description": "",
    "recommended_video_description": ""
  }
}
"""

# Collect the events
#results = []

# API call to replicate
for event in rep_api.run(
    "mistralai/mixtral-8x7b-instruct-v0.1",
    input={
        "top_k": 50,
        "top_p": 0.9,
        "prompt": f"<s>[INST] generate An heading, five (5) talking_points, a recommended_image_description and a recommended_video_description from this {user_input} [/INST]",
        "temperature": 0.6,
        "system_prompt": "YOU ARE AN AI ASSISTANT THAT CAN generate An heading, five (5) talking_points, a recommended_image_description and a recommended_video_description",
        "length_penalty": 1,
        "max_new_tokens": 1024,
        "presence_penalty": 0
    },
):

      event_data = json.loads(event.data)


# Format the results as JSON
#json_output = json.dumps(results, indent=4)
print(event_data)

AttributeError: 'str' object has no attribute 'data'

In [ ]:
print(str(event.data), end="")

AttributeError: 'str' object has no attribute 'data'

In [ ]:
import replicate
import json

# Example user input
user_input = "Today I want to talk about the various ways I can expand my Fruit (Udara) Business."

# Prompt template
prompt_template = """
{
  "user_input": "{user_input}",
  "output": {
    "heading": "",
    "talking_points": [],
    "recommended_image_description": "",
    "recommended_video_description": ""
  }
}
"""


# Collect the events
results = []

# API call to replicate
for event in rep_api.stream(
    "mistralai/mixtral-8x7b-instruct-v0.1",
    input={
        "top_k": 50,
        "top_p": 0.9,
        "prompt": f"<s>[INST] generate An heading, five (5) talking_points, a recommended_image_description and a recommended_video_description from this {user_input} [/INST]",
        "temperature": 0.6,
        "system_prompt": "YOU ARE AN AI ASSISTANT THAT CAN generate An heading, five (5) talking_points, a recommended_image_description and a recommended_video_description",
        "length_penalty": 1,
        "max_new_tokens": 1024,
        "presence_penalty": 0
    },
):
    results.append(event)

# Format the results as JSON
json_output = json.dumps(results, indent=4)
print(json_output)


TypeError: Object of type ServerSentEvent is not JSON serializable

In [ ]:
# Define the model and input prompt
model_name = "mistralai/mixtral-8x7b-instruct-v0.1"
input_prompt = "list names of 50 Christian Inventors?"

# Run the model
try:
    iterator = rep_api.run(
        model_name,
        input={"prompt": input_prompt, 'max_length': 500}
    )

    # Retrieve and print the results
    res_text_list = []
    for txt in iterator:
        res_text_list.append(txt)
    output_text = ''.join(res_text_list)
    print(output_text)
except replicate.exceptions.ReplicateError as e:
    print(f"ReplicateError: {e}")

I'm glad to assist you! Here are 50 Christian inventors who have made significant contributions in various fields. Please note that this list is not exhaustive and there are many other Christian inventors who have made important contributions throughout history.

1. Alexander Graham Bell - Inventor of the telephone
2. Tim Berners-Lee - Inventor of the World Wide Web
3. John Logie Baird - Inventor of the mechanical television
4. Guglielmo Marconi - Inventor of wireless telegraphy
5. Joseph Nicéphore


In [ ]:
def ask_user():
    user_input = input("Please enter something: ")
    print(f"You entered: {user_input}")


In [ ]:
# Call the function
ask_user()


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install -q -U langchain transformers bitsandbytes accelerate langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 71.2 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
model_4bit = AutoModelForCausalLM.from_pretrained( "mistralai/Mistral-7B-Instruct-v0.1", device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
device = "cuda" # the device to load the model onto

In [ ]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]


encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)

generated_ids = model_4bit.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

In [ ]:
pipeline_inst = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=2500,
        do_sample=True,
        top_k=15,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

llm = HuggingFacePipeline(pipeline=pipeline_inst)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [ ]:
# Define your desired data structure.
class Ke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [ ]:
# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Ke)

prompt = PromptTemplate(
    template="Generate one (1) Topic from this text.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser
chain.invoke({"query": joke_query})

In [ ]:
query = "i want to talk about different airlines and the cost and the country they visit"
result = llm(prompt.format(text=query))

In [ ]:
template = """
[INST] <>
Generate one (1) Topic from this text.
<>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [ ]:
query = "i want to talk about different airlines and the cost and the country they visit"
result = llm(prompt.format(text=query))

In [ ]:
result
#display(Markdown(f"<b>{query}</b>"))
#display(Markdown(f"<p>{result}</p>"))

'\n[INST] <>\nGenerate one (1) Topic from this text.\n<>\n\ni want to talk about different airlines and the cost and the country they visit [/INST]\nOne possible topic that could be generated from this text is "A Comparison of Airline Costs and Destinations."'

In [ ]:
template = """
[INST] <>
5 Blog Post Titles Generator from text.
<>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [ ]:
query = "A Comparison of Airline Costs and Destinations."
result = llm(prompt.format(text=query))
result

'\n[INST] <>\n5 Blog Post Titles Generator from text.\n<>\n\nA Comparison of Airline Costs and Destinations. [/INST]\n\n1. "5 Most Affordable Airlines: Which One is Right for You?"\n2. "From New York to Tokyo: A Guide to Airfare Costs Worldwide"\n3. "Exploring Cheap Alternatives to Major Airlines: A Comprehensive Review"\n4. "A World of Flights: Analyzing International Air Travel Costs"\n5. "Flying on a Budget: Best Practices and Tips for Travelers"'

In [ ]:
template = """
[INST] <>
Content Creator and Writer that generates Long articles.
<>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [ ]:
query = "The Key Elements of a Successful Investor Meeting"
result = llm(prompt.format(text=query))
result

"\n[INST] <>\nContent Creator and Writer that generates Long articles.\n<>\n\nThe Key Elements of a Successful Investor Meeting [/INST]\n\nAs an investor, attending a meeting with potential investors is an exciting and potentially life-changing opportunity. However, it can also be overwhelming and stressful if you're not well prepared. To ensure a successful investor meeting, here are the key elements you need to focus on.\n\n1. Clear and compelling pitch\n\nYour pitch should clearly explain your business, your market, and your competitive advantage. It should capture the investor's attention and convince them that your idea is worth pursuing. Practice your pitch until you can deliver it fluently and confidently.\n\n2. Financial projections\n\nProper financial projections are critical to convince investors that your business has potential for growth. Investors want to see that you have analyzed your revenue, expenses, and cash flow, and that you have realistic financial goals. Be prepa

In [ ]:
# Define the template for generating content
template = """<s>[INST] You are a respectful and helpful assistant, respond always be precise, assertive and politely answer in few words conversational English.
Answer the question below from context below :
{question} [/INST] </s>
"""

# Travel agency details dictionary
new_travel_agency_details_dict = {
    'brand_creator_user_id': '874nfd827sdf872nsd83274',
    'brand_creator_role': 'ceo_founder',  # other options are 'social_media_manager', 'content_marketer', 'growth_marketer', 'other_employee'
    'brand_id': 'travel12345',
    'brand_name': 'GlobeTrekker',
    'brand_handle': 'globetrekker',
    'brand_image_url': 'https://example.com/images/globetrekker_logo.png',
    'brand_type': 'travel_agency',  # this would be prefilled based on the user's selection at the previous page
    'brand_website_url': 'www.globetrekker.com',
    'brand_description': 'GlobeTrekker is a travel agency that specializes in creating personalized travel experiences. We offer curated travel packages, custom itineraries, and 24/7 customer support to ensure your journey is seamless and unforgettable.',
    'other': {}
}

In [ ]:
template = """<s>[INST] You are an respectful and helpful assistant, respond always be precise, assertive and politely answer in few words conversational english.
Answer the question below from context below :
{question} [/INST] </s>
"""

def generate_response(question):
  prompt = PromptTemplate(template=template, input_variables=["question","context"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.run({"question":question})
  return response

In [ ]:
# Define the template for generating a title
title_template = """<s>[INST] Generate a catchy and engaging title based on the following user input:
{user_input} [/INST] </s>"""

# Function to generate title
def generate_title(user_input):
    prompt = PromptTemplate(template=title_template, input_variables=["user_input"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    response = llm_chain.run({"user_input": user_input})
    # Extract title from response
    title = response.strip().replace("<s>", "").replace("</s>", "").split(":")[-1].strip()
    return title

In [ ]:
generate_title("i want to talk about the different airlines and the cost and the country they visit")

'i want to talk about the different airlines and the cost and the country they visit [/INST]'

In [ ]:
talking_points_template = """<s>[INST] Generate 5 talking points based on the following user input:
{user_input} [/INST] </s>"""

story_template = """<s>[INST] Generate a detailed story based on the following talking point:
{talking_point} [/INST] </s>"""

# Combined function
def generate_content(user_input):
    # Generate title
    title_prompt = PromptTemplate(template=title_template, input_variables=["user_input"])
    llm_chain = LLMChain(prompt=title_prompt, llm=llm)
    title_response = llm_chain.run({"user_input": user_input})
    title = title_response.strip().replace("<s>", "").replace("</s>", "").split(":")[-1].strip()

    return title

    # Generate talking points
    tp_prompt = PromptTemplate(template=talking_points_template, input_variables=["user_input"])
    llm_chain = LLMChain(prompt=tp_prompt, llm=llm)
    tp_response = llm_chain.run({"user_input": user_input})
    talking_points = [tp.strip() for tp in tp_response.strip().replace("<s>", "").replace("</s>", "").split('\n') if tp.strip()]
    talking_points = talking_points[:5]

    return talking_points
    #print("Generated Talking Points:")
    for i, point in enumerate(talking_points, 1):
        print(f"{i}. {point}")

    # Use one of the talking points to generate a story
    if talking_points:
        chosen_talking_point = talking_points[0]
        story_prompt = PromptTemplate(template=story_template, input_variables=["talking_point"])
        llm_chain = LLMChain(prompt=story_prompt, llm=llm)
        story_response = llm_chain.run({"talking_point": chosen_talking_point})
        story = story_response.strip().replace("<s>", "").replace("</s>", "").split(":")[-1].strip()

        return story


In [ ]:
generate_content("A Comparative Analysis of Airlines and Their Global Reach")

'A Comparative Analysis of Airlines and Their Global Reach [/INST]'

In [ ]:
def generate_response(question, context):
    prompt = PromptTemplate(template=template, input_variables=["question", "context"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    response = llm_chain.run({"question": question, "context": context})
    return response

In [ ]:
template = """<s>[INST] You are an respectful and helpful assistant, respond always be precise, assertive and politely answer in few words conversational english.
Answer the question below from context below :
{question} [/INST] </s>
"""

def generate_response(question):
  prompt = PromptTemplate(template=template, input_variables=["question","context"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.run({"question":question})
  return response

In [ ]:
generate_response("list 16 innovators/inventors that believe in God")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Travel agency details dictionary
new_travel_agency_details_dict = {
    'brand_creator_user_id' : '874nfd827sdf872nsd83274',
    'brand_creator_role' : 'ceo_founder',  # other options are 'social_media_manager', 'content_marketer', 'growth_marketer', 'other_employee'
    'brand_id' : 'travel12345',
    'brand_name' : 'GlobeTrekker',
    'brand_handle' : 'globetrekker',
    'brand_image_url' : 'https://example.com/images/globetrekker_logo.png',
    'brand_type' : 'travel_agency',  # this would be prefilled based on the user's selection at the previous page
    'brand_website_url' : 'www.globetrekker.com',
    'brand_description' : 'GlobeTrekker is a travel agency that specializes in creating personalized travel experiences. We offer curated travel packages, custom itineraries, and 24/7 customer support to ensure your journey is seamless and unforgettable.',
    'other' : {}
}

In [ ]:
i want to talk about the different airlines and the cost and the country they visit

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the tokenizer and model
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set the model to evaluation mode and use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

In [ ]:
# Travel agency details dictionary
new_travel_agency_details_dict = {
    'brand_creator_user_id' : '874nfd827sdf872nsd83274',
    'brand_creator_role' : 'ceo_founder',
    'brand_id' : 'travel12345',
    'brand_name' : 'GlobeTrekker',
    'brand_handle' : 'globetrekker',
    'brand_image_url' : 'https://example.com/images/globetrekker_logo.png',
    'brand_type' : 'travel_agency',  # this would be prefilled based on the user's selection at the previous page
    'brand_website_url' : 'www.globetrekker.com',
    'brand_description' : 'GlobeTrekker is a travel agency that specializes in creating personalized travel experiences. We offer curated travel packages, custom itineraries, and 24/7 customer support to ensure your journey is seamless and unforgettable.',
    'other' : {}
}

def generate_travel_content(travel_agency_details):
    # Extract relevant details to provide context to the AI
    agency_info = (
        f"Brand Name: {travel_agency_details['brand_name']}\n"
        f"Brand Description: {travel_agency_details['brand_description']}\n"
        f"Website: {travel_agency_details['brand_website_url']}\n\n"
    )

    # Ask the travel agency for input
    user_input = input("Please describe the type of travel content you would like to generate: ")

    # Combine the agency info with the user input
    prompt = agency_info + "User Request: " + user_input

    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate content
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=150,  # Adjust the max_length as needed
            num_return_sequences=1,
            temperature=0.7,  # Adjust temperature for creativity
            top_k=50,        # Adjust top_k for more/less randomness
            top_p=0.95       # Adjust top_p for more/less randomness
        )

    # Decode and print the generated content
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\nGenerated Travel Content:\n")
    print(generated_text)

# Call the function to generate content with travel agency details
generate_travel_content(new_travel_agency_details_dict)


In [ ]:
class TravelContentGenerator:
    def __init__(self):

        self.llm = HuggingFacePipeline(pipeline=pipeline_inst)

        # Define the templates
        self.title_template = """<s>[INST] Generate a catchy and engaging title based on the following user input:
        {user_input} [/INST] </s>"""

        self.talking_points_template = """<s>[INST] Generate 5 talking points based on the following title:
        {title} [/INST] </s>"""

        self.article_template = """<s>[INST] Generate a detailed story based on the following talking point:
        {talking_point} [/INST] </s>"""

    def generate_title(self, user_input):
        # Generate title
        title_prompt = PromptTemplate(template=self.title_template, input_variables=["user_input","context"])
        llm_chain = LLMChain(prompt=title_prompt, llm=self.llm)
        title_response = llm_chain.run({"user_input": user_input})
        title = title_response.strip().replace("<s>", "").replace("</s>", "").replace("[INST]", "").replace("[/INST]", "").split(":")[-1].strip()
        return title

    def generate_talking_points(self, title):
        # Generate talking points
        tp_prompt = PromptTemplate(template=self.talking_points_template, input_variables=["title","context"])
        llm_chain = LLMChain(prompt=tp_prompt, llm=self.llm)
        tp_response = llm_chain.run({"title": title})
        talking_points = [tp.strip().replace("[INST]", "").replace("[/INST]", "") for tp in tp_response.strip().replace("<s>", "").replace("</s>", "").split('\n') if tp.strip()]
        talking_points = talking_points[:5]
        return talking_points

    def generate_article(self, talking_point):
        # Generate article
        story_prompt = PromptTemplate(template=self.article_template, input_variables=["talking_point","context"])
        llm_chain = LLMChain(prompt=story_prompt, llm=self.llm)
        story_response = llm_chain.run({"talking_point": talking_point})
        story = story_response.strip().replace("<s>", "").replace("</s>", "").replace("[INST]", "").replace("[/INST]", "").split(":")[-1].strip()
        return story

# NEW

In [ ]:
import torch
import gradio as gr

from textwrap import fill
from IPython.display import Markdown, display

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    )

from langchain import PromptTemplate
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

In [ ]:
llm = HuggingFacePipeline(
    pipeline=pipeline,
    )

In [ ]:
query = "Explain the difference between ChatGPT and open source LLMs in a couple of lines."
result = llm(
    query
)

display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))